<a href="https://colab.research.google.com/github/alb3rrt/tfm_unir_afernandez/blob/main/ArxiV_Search_Engine_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import defaultdict
from pathlib import Path
import pandas as pd
import spacy
import tqdm
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv("/content/drive/MyDrive/TFM/Albert/Data.csv")

In [4]:
data.head()

,categories,abstract
0,['physics'],fully differential calculation perturbative qu...
1,"['computer science', 'math']",describe new algorithm ell pebble game colors ...
2,['physics'],evolution earth moon system described dark mat...
3,['math'],show determinant stirling cycle numbers counts...
4,['math'],paper show compute lambda alpha norm alpha ge ...


In [5]:
#1738 samples

pow = "/content/drive/MyDrive/TFM/Albert/ArXiv_Papers"

results = defaultdict(list)
for file in Path(pow).iterdir():
    with open(file, "r",encoding='ISO 8859-1') as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())

In [6]:
#pruebas

#pow = "/content/drive/MyDrive/TFM/Albert/ArXiv_Papers"

#results = defaultdict(list)
#for file in Path(pow).iterdir():
#    with open(file, "r",encoding='ISO 8859-1') as file_open:
#        lines = file_open.readlines()
#        text=''.join(lines)
#        sentences = text.split('.')
#        results["file_name"].append(file.name)
#        results["text"].append(sentences)

In [7]:
print (results["text"][1])

Astronomy & Astrophysics manuscript no. h2c5_tmc1_Cabezas June 2, 2021

©ESO 2021

arXiv:2106.00635v1 [astro-ph.GA] 1 Jun 2021

Cumulene carbenes in TMC-1: Astronomical discovery of l-H2C5
C. Cabezas1, B. Tercero2, 3, M. Agúndez1, N. Marcelino1, J. R. Pardo1, P. de Vicente3 and J. Cernicharo1
1 Grupo de Astrofísica Molecular, Instituto de Física Fundamental (IFF-CSIC), C/ Serrano 121, 28006 Madrid, Spain. e-mail: carlos.cabezas@csic.es; jose.cernicharo@csic.es
2 Observatorio Astronómico Nacional (IGN), C/ Alfonso XII, 3, 28014, Madrid, Spain. 3 Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain.
Received; accepted
ABSTRACT
We report the first detection in space of the cumulene carbon chain l-H2C5. A total of eleven rotational transitions, with Jup = 7-10 and Ka = 0 and 1, were detected in TMC-1 in the 31.0-50.4 GHz range using the Yebes 40m radio telescope. We derive a column density of (1.8±0.5)×1010 cm-2. In addition, we report observatio

In [8]:
df = pd.DataFrame(results)

In [9]:
nlp = spacy.load("en_core_web_sm")
stops = spacy.lang.en.stop_words.STOP_WORDS


def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)



In [10]:
df['text_clean']=df['text'].progress_apply(normalize,lowercase=True, remove_stopwords=True)

100%|██████████| 1768/1768 [52:44<00:00,  1.79s/it]


In [11]:
df

,file_name,text,text_clean
0,2106.0210_Improving_Formality_Style_Transfer_w...,Improving Formality Style Transfer with Contex...,improve formality style transfer context - awa...
1,2106.0635_Cumulene_carbenes_in_TMC-1__Astronom...,Astronomy & Astrophysics manuscript no. h2c5_t...,astronomy & astrophysics manuscript . h2c5_tmc...
2,2106.01417_Towards_nonlinear_axion-dilaton_ele...,arXiv:2106.01417v1 [gr-qc] 2 Jun 2021\n\nTowar...,arxiv:2106.01417v1 [ gr - qc ] 2 jun 2021 nonl...
3,2106.01039_Classification_of_OGLE_eclipsing_bi...,"Draft version June 3, 2021 Typeset using LATEX...","draft version june 3 , 2021 typeset use latex ..."
4,"2106.01137_Cosmological_evolution_in_$f(T,B)$_...",arXiv:2106.01137v1 [gr-qc] 2 Jun 2021\n\nCosmo...,arxiv:2106.01137v1 [ gr - qc ] 2 jun 2021 cosm...
...,...,...,...
1763,2106.0532_Topology_and_Admittance_Estimation__...,1\nTopology and Admittance Estimation: Precisi...,1 topology admittance estimation : precision l...
1764,2106.01828_Ultrafast_Electron_Dynamics_in_Magn...,Ultrafast Electron Dynamics in Magnetic Thin F...,ultrafast electron dynamic magnetic thin film ...
1765,2106.0473_SemEval-2021_Task_1__Lexical_Complex...,SemEval-2021 Task 1: Lexical Complexity Predic...,semeval-2021 task 1 : lexical complexity predi...
1766,2106.01354_multiPRover__Generating_Multiple_Pr...,MULTIPROVER: Generating Multiple Proofs for Im...,multiprover : generate multiple proof improved...


In [12]:
#df['text_clean']=df['text_clean'].str.join(' ')

In [13]:
category_map = {'acc-phys':'Physics',
                'adap-org':'Physics',
                'alg-geom':'Maths',
                'ao-sci':'Physics',
                'astro-ph':'Physics',
                'atom-ph':'Physics', 
                'bayes-an':'Maths', 
                'chao-dyn':'Physics', 
                'chem-ph':'Physics', 
                'cmp-lg':'Computer Science', 
                'comp-gas':'Computer Science',
                'cond-mat':'Physics', 
                'cond-mat':'Physics',
                'cs':'Computer Science', 
                'dg-ga':'Computer Science',
                'econ':'Economics', 
                'eess':'Electrical Engineering and Systems Science', 
                'funct-an':'Maths',
                'gr-qc':'Physics',
                'hep-ex':'Physics',
                'hep-lat':'Physics',
                'hep-ph':'Physics', 
                'hep-th':'Physics', 
                'math':'Maths', 
                'math-ph':'Maths',
                'mtrl-th':'Maths',
                'nlin':'Physics', 
                'nucl-ex':'Physics',
                'nucl-th':'Physics', 
                'patt-sol':'Computer Science',
                'physics':'Physics',
                'plasm-ph':'Physics',
                'q-alg':'Maths', 
                'q-bio':'Quantitative Biology', 
                'q-fin':'Quantitative Finance', 
                'quant-ph':'Physics',
                'solv-int':'Maths',
                'stat':'Statistics',
                'supr-con':'Physics'}

In [14]:
model=keras.models.load_model("/content/drive/MyDrive/TFM/Albert/OUTPUT_NAME/Paper_Classifier")

with open(r"/content/drive/MyDrive/TFM/Albert/tokenizer.pickle", "rb") as input_file:
  token = pickle.load(input_file)




#df.to_csv("Cleaned_Output.csv",index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
results=token.inverse_transform(np.argmax(model.predict(df['text_clean']), axis=-1))


In [16]:
rev_subs = { v:k for v,k in category_map.items()}
results=[rev_subs.get(item,item)  for item in results]
print (results)

['Computer Science', 'Quantitative Biology', 'Physics', 'Computer Science', 'Physics', 'Physics', 'Electrical Engineering and Systems Science', 'Computer Science', 'Physics', 'Computer Science', 'Electrical Engineering and Systems Science', 'Maths', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Computer Science', 'Electrical Engineering and Systems Science', 'Computer Science', 'Maths', 'Computer Science', 'Physics', 'Physics', 'Computer Science', 'Maths', 'Computer Science', 'Computer Science', 'Computer Science', 'Physics', 'Computer Science', 'Maths', 'Maths', 'Computer Science', 'Computer Science', 'Physics', 'Maths', 'Physics', 'Maths', 'Computer Science', 'Computer Science', 'Electrical Engineering and Systems Science', 'Computer Science', 'Physics', 'Electrical Engineering and Systems Science', 'Maths', 'Maths', 'Computer Science', 'Physics', 'Physics', 'Computer Science', 'Computer Sc

In [17]:
df['category']=results
df[df['file_name']=='2106.01174_Nitsche_s_Finite_Element_Method_for_Model_Coupling_in_Elasticity.pdf.txt']

#df['file_name'].unique()

,file_name,text,text_clean,category
293,2106.01174_Nitsche_s_Finite_Element_Method_for...,arXiv:2106.01174v1 [math.NA] 2 Jun 2021\n\nNit...,arxiv:2106.01174v1 [ math.na ] 2 jun 2021 nits...,Maths


In [18]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model_bert = SentenceTransformer('bert-base-nli-mean-tokens')

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 2.8 MB 19.2 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 636 kB 76.5 MB/s 
     |████████████████████████████████| 3.3 MB 68.2 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=7e9dd826f05348501e957df3c68f73e8db364e36959cf58e1032646f26f979f5
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
import scipy
query = "Lithium is an important element for the understanding of ultracool dwarfs because it is lost to fusion at masses above  68 MJ. Hence, the presence or absence of atomic Li has served as an indicator of the nearby H-burning boundary at about 75 MJ between brown-dwarfs and very low-mass stars. Historically the \"Lithium test\", a search for the presence and strength of the Li line at 670.8 nm, has been a marker if an object has a substellar mass with stellar-like spectral energy distribution (e.g., a late-type M dwarf). While the Li test could in principle also be used to distinguish masses of later-type L-T dwarfs, Li is predominantly no longer found as an atomic gas, but rather a molecular species such as LiH, LiF, LiOH, and LiCl in their cooler atmospheres. L- and T-type brown dwarfs are also quite faint at 670 nm and thus challenging targets for high resolution spectroscopy. But only recently have experimental molecular line lists become available for the molecular Li species, allowing molecular Li mass discrimination. In this study, we generated the latest opacity of each of these Li-bearing molecules and performed thermochemical equilibrium atmospheric composition calculation of the abundance of these molecules. Finally, we computed thermal emission spectra for a series of radiativeconvective equilibrium models of cloudy and cloudless brown dwarf atmospheres (with Teff = 500\u22602400 K, and log g=4.0, 4.5, 5.0) to understand where the presence or absence of atmospheric lithium-bearing species is most easily detected as a function of brown dwarf mass and age. After atomic Li, the best spectral signatures were found to be LiF at 10.5 - 12.5 \xB5m and LiCl at 14.5 - 18.5 \xB5m. LiH also shows a narrow feature at  9.38 \xB5m.  1. INTRODUCTION More than 25 years ago, the observation of the first brown dwarfs, Teide 1 and Gliese 229B, ushered a new era in astronomy (Rebolo et al. 1995; Oppenheimer et al. 1995; Nakajima et al. 1995). Since then, many models and observations have been done to understand the chemistry and physics of brown dwarf atmospheres, their atmospheric evolution with age, mass, and effective temperature (e.g., Saumon & Marley 2008; Chabrier et al. 2000; Burrows et al. 1997), as well as their connection to the planet and stellar main characteristics (e.g., Marley & Robinson 2015; Zhang 2020). Luminosity, age, mass, radius, surface gravity, atmospheric evolution, effective temperature, and cloud/aerosol formation are among the main active open questions waiting to be addressed to better understand these objects.  NASA Postdoctoral Fellow corresponding author: e.gharibnezhad@asu.edu  One indicator used to distinguish between stars and brown dwarfs is the detection of the atomic lithium resonance line at 670.8 nm, which was proposed by Rebolo et al. (1992) and aptly named the \"lithium test\". Atomic Li is converted to He through fusion at central temperatures of  2 - 3 \u25CA 106 K. Thus objects which are not massive enough to reach such temperatures in their interior do not burn Li. Nelson et al. (1993) and Pozio (1991) calculated a transition mass of 62 - 65 MJ for the Li burning. Magazzu et al. (1993) showed that the required mass to burn Li by half is 84 and M = 63 MJ for 60 Myr and 250 Myr objects, respectively. Similarly, Chabrier et al. (1996) modeled the depletion of Li along with mass and age, and found that the initial lithium burns by a factor of 2 over 0.26 Gyr and by a factor of 100 over  1 Gyr for 64 MJ object. In addition, B"#@param {type: "string"}

query=normalize(query,lowercase=True, remove_stopwords=True)
queries = [query]
query_embeddings = model_bert.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches =  10#@param {type: "number"}

dropdown = 'Maths' #@param ['Physics','Maths','Economics','Computer Science','Quantitative Biology','Quantitative Finance','Statistics','Electrical Engineering and Systems Science','Economics']

sentences = df['text'][df['category']==dropdown].values.tolist()

sentence_embeddings = model_bert.encode(sentences)
print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    #print(type(results))

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {0} files where the input matches the corpus for {1} category are:".format(number_top_matches,dropdown))
    print("\n\n======================\n\n")
    for idx, distance in results[0:number_top_matches]:
      if (distance)>0.5:
          print(df['file_name'].iloc[[idx]].to_list(),"(Cosine Score: %.4f)" % (distance))
          #print(sentences[idx].strip())
          #print()
          print("\n\n======================\n\n")

Semantic Search Results




Query: lithium important element understanding ultracool dwarf -PRON- lose fusion masse 68 mj . , presence absence atomic li serve indicator nearby h - burn boundary 75 mj brown - dwarfs low - mass star . historically " lithium test " , search presence strength li line 670.8 nm , marker object substellar mass stellar - like spectral energy distribution ( e.g. , late - type m dwarf ) . li test principle use distinguish masse later - type l - t dwarfs , li predominantly longer find atomic gas , molecular specie lih , lif , lioh , licl -PRON- cool atmosphere . l- t - type brown dwarf faint 670 nm challenge target high resolution spectroscopy . recently experimental molecular line list available molecular li specie , allow molecular li mass discrimination . study , -PRON- generate late opacity li - bear molecule perform thermochemical equilibrium atmospheric composition calculation abundance molecule . finally , -PRON- compute thermal emission spectra series rad

In [20]:
sentence_embeddings.shape
#print(query_embeddings)
print(sentences[1][3:45])
#a = df['text_clean'][df['category']==dropdown].values.tolist()
a = df['text_clean'][df['category']==dropdown].values.tolist()
print(a[1])
print(a[2])
print(a[3])
#print(len(query_embeddings))
#print(len(sentences))

iv:2106.00535v1 [math.AC] 30 May 2021

The
arxiv:2106.00535v1 [ math.ac ] 30 2021 s - weak global dimension commutative ring xiaolei zhang department basic course , chengdu aeronautic polytechnic , chengdu 610100 , china e - mail : zxlrghj@163.com abstract paper , -PRON- introduce study s - weak global dimension s - w.gl.dim(r ) commutative ring r multiplicative subset s r. , commutative ring s - weak global dimension 1 study . finally , -PRON- investigate s - weak global dimension factor ring polynomial ring . key word : s - flat module , s - flat dimension , s - weak global dimension . 2010 mathematic subject classification : 13d05 , 13d07 , 13b25 . article , r commutative ring unity 1 s multiplicative subset r , , 1 s s1s2 s s1 s , s2 s. 2002 , anderson dumitrescu [ 1 ] define s - noetherian ring r ideal r s - finite . recall [ 1 ] r - module m s - finite provide sm f s s finitely generate submodule f m. r - module t uniformly s - torsion st = 0 s s [ 11 ] . r - module m s - finite 